# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report


In [2]:
path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [3]:
STATE_VACCINE = {}

In [5]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)

Enter the date of collecting data (Today's data in YYYY-MM-DD):


 2021-04-28


Hello, 2021-04-28


# V. Tableau States

## D. New York (Done)


Solution is from

https://github.com/bertrandmartel/tableau-scraping/issues/9

In [6]:
from tableauscraper import TableauScraper as TS

url = 'https://covid19tracker.health.ny.gov/views/Race_Ethnicity_Public/RacebyCounty'

ts = TS()
ts.loads(url)

workbook = ts.getWorkbook()

parameters = workbook.getParameters()
print(parameters)

ts = TS()
ts.loads(url)

# set parameters column / value
workbook = workbook.setParameter('Show Value as', "Number")
# display worksheets
Data = workbook.getWorksheet('Race').data

[{'column': 'Show Value as', 'values': ['Number', 'Percentage'], 'parameterName': '[Parameters].[Parameter 1]'}]


In [7]:
import pandas as pd

L = []

for county, df in Data[Data['Measure Names-alias'] == 'People with at least one vaccine dose '].groupby('County-alias'):
    county, df
    # print(county)
    if county == '%all%':
        continue
    D = {}
    D['County'] = county
    D['Date'] = Today_Date
    for k, row in df.iterrows():
        d = row.to_dict()
        # print(d)
        Race = d['Race-value'] +'_Num'
        Numb = d['Measure Values-alias']
        Numb = int(Numb.replace(',',''))
        D[Race] = Numb

    L.append(D)

NewYork = pd.DataFrame(L)
NewYork

,County,Date,Other_Num,Asian_Num,African American_Num,White_Num
0,Albany,2021-04-28,1113.0,10478,10828,108568
1,Allegany,2021-04-28,55.0,130,75,9334
2,Bronx,2021-04-28,9350.0,26861,111698,81306
3,Broome,2021-04-28,670.0,3971,2457,72495
4,Cattaraugus,2021-04-28,541.0,182,200,19422
...,...,...,...,...,...,...
57,Washington,2021-04-28,88.0,151,132,21149
58,Wayne,2021-04-28,197.0,233,663,31698
59,Westchester,2021-04-28,5046.0,31443,39839,306708
60,Wyoming,2021-04-28,37.0,79,53,12069


In [8]:
df

,Race-value,Race-alias,Region-value,Region-alias,County-value,County-alias,Measure Names-value,Measure Names-alias,Measure Values-alias
128,Other,Other,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,15
138,Asian,Asian,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,25
148,African American,African American,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,35
158,White,White,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,"8,287"


In [9]:
# print(len(L))

import numpy as np


def convert_integer(v):
    if type(v) == str:
        v = float(int(v.replace(',', '')))
    return v



state = 'New York' ## 

new_L = []

for idx, row in NewYork.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str or d['County'] == 'Total' :
        print(d)
        continue 
        
    # try:
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



    new_d['Numb_White'] = convert_integer(d['White_Num'])
    # new_d['Rate_White'] = convert_percentage(d['White_Rate']) #  if type(d['White_Rate']) == str else d['White_Rate'] 

    new_d['Numb_Black'] = convert_integer(d['African American_Num'])
    # new_d['Rate_Black'] = convert_percentage(d['African American_Rate'])

    new_d['Numb_Asian'] = convert_integer(d['Asian_Num'])
    # new_d['Rate_Asian'] = convert_percentage(d['Asian_Rate'])

    new_d['Date'] = d['Date']

    new_L.append(new_d)

#     except:
#         print(d)

    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head() # Allogany

(62, 22)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_AsianC,Rate_Black,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New York,Albany,"Albany, New York",2021-04-28,10478,10828,108568,0.501700,0.434880,0.251253,...,0.453470,930,2401,20885,24094,43096,48463,231538,239416,305506
1,New York,Allegany,"Allegany, New York",2021-04-28,130,75,9334,0.225303,0.163522,0.112952,...,0.209193,147,324,577,795,664,964,44048,44619,46091
2,New York,Bronx,"Bronx, New York",2021-04-28,26861,111698,81306,0.415252,0.376215,0.180579,...,0.120325,41319,59291,64686,71398,618556,663574,634280,675719,1418207
3,New York,Broome,"Broome, New York",2021-04-28,3971,2457,72495,0.471391,0.390654,0.202790,...,0.429259,519,1680,8424,10165,12116,15644,163473,168884,190488
4,New York,Cattaraugus,"Cattaraugus, New York",2021-04-28,182,200,19422,0.282171,0.194237,0.178094,...,0.271728,2742,3292,645,937,1123,2025,69919,71476,76117


In [10]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,62.0,0.187432,0.070013,0.023799,0.137511,0.204021,0.228142,0.438965
1,Rate_White,62.0,0.378945,0.082246,0.128186,0.324558,0.372565,0.436309,0.627354
2,Rate_Parity,62.0,0.191513,0.086130,-0.052392,0.142998,0.191469,0.243805,0.399788


## D. Wisconsin (Done)

New Method: https://github.com/bertrandmartel/tableau-scraping/issues/5

https://www.dhs.wisconsin.gov/covid-19/vaccine-data.htm#day


In [11]:

from tableauscraper import TableauScraper as TS

# url = "https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents/VaccinatedWisconsin-County"
url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16129838459350/VaccinatedWisconsin-County'

ts = TS()
ts.loads(url)

worksheet = ts.getWorksheet("Map")

SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}
[i for i in SelectionDict]

['County',
 'SUM(Number With One Dose)',
 'SUM(Number With Two Doses)',
 'SUM(Popcount)',
 'AGG(Number complete for map)',
 'Latitude (generated)',
 'Longitude (generated)',
 'AGG(% complete)',
 'AGG(Calc- Initiation or Full Coverage)',
 'AGG(% at least 1 dose)']

In [12]:
dashboard = worksheet.select("County", "Price County")
df = dashboard.getWorksheet("Race vax/unvax county").data
df

,Race-value,Race-alias,SUM(Initiation or completed count for TT)-alias,AGG(min(1))-value,AGG(min(1))-alias,ATTR(initiation or completed text for TT)-alias,AGG(Geography TT)-alias,AGG(Calc- Initiation or Full Coverage)-alias,Measure Names-alias
0,1,White,4990,1,1,have received at least one dose,Price County,0.388387,min(1)
1,2,Black,12,1,1,have received at least one dose,Price County,0.082192,min(1)
2,4,Asian,34,1,1,have received at least one dose,Price County,0.151111,min(1)
3,3,American Indian,37,1,1,have received at least one dose,Price County,0.280303,min(1)


In [13]:
def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Race-alias']
        D['Numb_' + race] = d['SUM(Initiation or completed count for TT)-alias']
        D['Rate_' + race] = d['AGG(Calc- Initiation or Full Coverage)-alias']
    return D

county = "Price County"
df = df

process_county(county, df)

{'County': 'Price',
 'Numb_White': 4990,
 'Rate_White': 0.388387297633873,
 'Numb_Black': 12,
 'Rate_Black': 0.0821917808219178,
 'Numb_Asian': 34,
 'Rate_Asian': 0.1511111111111111,
 'Numb_American Indian': 37,
 'Rate_American Indian': 0.2803030303030303}

In [14]:


def get_Wisconsin_Raw():
    
    # url = "https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents/VaccinatedWisconsin-County"
    url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16129838459350/VaccinatedWisconsin-County'


    ts = TS()
    ts.loads(url)
    worksheet = ts.getWorksheet("Map")
    SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}
    
    L = []

    for county in SelectionDict['County']:
        try:
            dashboard = ts.getWorksheet("Map").select("County", county)
            df = dashboard.getWorksheet("Race vax/unvax county").data
            df_dict = process_county(county, df)
            L.append(df_dict)
            print('Success\t\t', county)


        except:
            print('Fail', county)

    
    Wisconsin = pd.DataFrame(L)
    
    return Wisconsin


Wisconsin = get_Wisconsin_Raw()

Success		 Wood County
Success		 Winnebago County
Success		 Waushara County
Success		 Waupaca County
Success		 Waukesha County
Success		 Washington County
Success		 Washburn County
Success		 Walworth County
Success		 Vilas County
Success		 Vernon County
Success		 Trempealeau County
Success		 Taylor County
Success		 St croix County
Success		 Sheboygan County
Success		 Shawano County
Success		 Sawyer County
Success		 Sauk County
Success		 Rusk County
Success		 Rock County
Success		 Richland County
Success		 Racine County
Success		 Price County
Success		 Portage County
Success		 Polk County
Success		 Pierce County
Success		 Pepin County
Success		 Ozaukee County
Success		 Outagamie County
Success		 Oneida County
Success		 Oconto County
Success		 Monroe County
Success		 Milwaukee County
Success		 Menominee County
Success		 Marquette County
Success		 Marinette County
Success		 Marathon County
Fail Manitowoc County
Success		 Lincoln County
Success		 Langlade County
Success		 Lafayette County
S

In [15]:
# Wisconsin.to_csv('FinalResult/Raw_Wisconsin.csv')
print(Wisconsin.shape)
Wisconsin.head()

(71, 9)


,County,Numb_White,Rate_White,Numb_Black,Rate_Black,Numb_Asian,Rate_Asian,Numb_American Indian,Rate_American Indian
0,Wood,25912,0.372032,118,0.114230,528,0.350831,123,0.151665
1,Winnebago,59675,0.374051,550,0.104882,1767,0.311256,293,0.202208
2,Waushara,6508,0.277408,26,0.042414,34,0.220779,20,0.092593
3,Waupaca,16361,0.328594,49,0.114754,89,0.276398,79,0.175556
4,Waukesha,168601,0.446781,2113,0.250563,6901,0.410554,563,0.354534


In [16]:
state = 'Wisconsin'


def standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION):
    
    state = 'Wisconsin' ## 

    new_L = []

    for idx, row in Wisconsin.iterrows(): ## 
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
        

        RenameCounties = {
            'Fond Du Lac': 'Fond du Lac',
            'St Croix': 'St. Croix',
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']


        new_d['Numb_White'] = d['Numb_White']
        new_d['Rate_White'] = d['Rate_White']

        new_d['Numb_Black'] = d['Numb_Black']
        new_d['Rate_Black'] = d['Rate_Black']

        new_d['Numb_Asian'] = d['Numb_Asian']
        new_d['Rate_Asian'] = d['Rate_Asian']

        new_d['Numb_AIndA'] = d['Numb_American Indian']
        new_d['Rate_AIndA'] = d['Rate_American Indian']

        new_L.append(new_d)

    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:

        Rate = T['Rate_' + Race] 
        T['Rate_' + Race + 'C'] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


    return Data, T


    
    
Data, T = standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION)

# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()


# diagnose_Rate(T)


(71, 24)


,State,County,Geographic Area Name,Numb_AIndA,Numb_Asian,Numb_Black,Numb_White,Rate_AIndA,Rate_AIndAC,Rate_Asian,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Wisconsin,Wood,"Wood, Wisconsin",123,528,118,25912,0.151665,0.151665,0.350831,...,0.372032,726,1036,1448,1660,702,1195,69122,70071,72999
1,Wisconsin,Winnebago,"Winnebago, Wisconsin",293,1767,550,59675,0.202208,0.202208,0.311256,...,0.374051,1314,2190,5258,6180,4302,5850,157796,160790,171907
2,Wisconsin,Waushara,"Waushara, Wisconsin",20,34,26,6508,0.092593,0.092593,0.220779,...,0.277408,193,350,133,186,521,644,23264,23568,24443
3,Wisconsin,Waupaca,"Waupaca, Wisconsin",79,89,49,16361,0.175556,0.175556,0.276398,...,0.328594,398,698,299,442,275,494,49381,49989,50990
4,Wisconsin,Waukesha,"Waukesha, Wisconsin",563,6901,2113,168601,0.354534,0.354534,0.410554,...,0.446781,1358,2854,15536,18090,7091,9611,373770,379839,404198


In [17]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,71.0,0.110026,0.057871,0.000000,0.070707,0.103604,0.134156,0.363388
1,Rate_White,71.0,0.371369,0.066522,0.216824,0.337027,0.366954,0.402329,0.573630
2,Rate_Parity,71.0,0.261343,0.067762,0.099770,0.221523,0.252192,0.296590,0.573630


## D. Ohio (Done)

https://coronavirus.ohio.gov/wps/portal/gov/covid-19/dashboards/covid-19-vaccine/covid-19-vaccination-dashboard

In [18]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)
dashboard = ts.getWorkbook()

countryDashboard = ts.getWorksheet("New Map").select("county", 'Wyandot')

raceDashboard = dashboard.setParameter('Key Metrics', "Race")

df = raceDashboard.getWorksheet('New Demographics').data
df

,Selected Demographic-value,Selected Demographic-alias,pop_pct_by_demographic-value,pop_pct_by_demographic-alias,SUM(Chosen Dose Status Metric)-value,SUM(Chosen Dose Status Metric)-alias
0,White,White,0.288664,0.288664,6124,6124
1,Unknown,Unknown,%null%,NA,970,970
2,Other,Other,%null%,NA,357,357
3,Black or African American,Black or African American,0.16092,0.16092,14,14
4,Asian,Asian,0.267974,0.267974,41,41
5,American Indian or Alaska Native,American Indian Alaska Native,0.153846,0.153846,10,10
6,Native Hawaiian or Other Pacific Islander,Native Hawaiian Pacific Islander,0.875,0.875,7,7
7,Multiracial,Multiracial,0.27459,0.27459,67,67


In [19]:
def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Selected Demographic-value']
        D['Numb_' + race] = d['SUM(Chosen Dose Status Metric)-value']
        D['Rate_' + race] = d['pop_pct_by_demographic-value']
    return D

county = 'Williams'
d = process_county(county, df)

sum([v for k, v in d.items() if 'Rate' in k and type(v) == float])
    

2.020993395577146

In [20]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)
workbook  = ts.getWorkbook()
worksheet = ts.getWorksheet("New Map")

counties = [
    t["values"] for t in worksheet.getSelectableItems()
    if t["column"] == "county"
][0]


print(counties)



['Wyandot', 'Wood', 'Williams', 'Wayne', 'Washington', 'Warren', 'Vinton', 'Van Wert', 'Union', 'Tuscarawas', 'Trumbull', 'Summit', 'Stark', 'Shelby', 'Seneca', 'Scioto', 'Sandusky', 'Ross', 'Richland', 'Putnam', 'Preble', 'Portage', 'Pike', 'Pickaway', 'Perry', 'Paulding', 'Ottawa', 'Noble', 'Muskingum', 'Morrow', 'Morgan', 'Montgomery', 'Monroe', 'Miami', 'Mercer', 'Meigs', 'Medina', 'Marion', 'Mahoning', 'Madison', 'Lucas', 'Lorain', 'Logan', 'Licking', 'Lawrence', 'Lake', 'Knox', 'Jefferson', 'Jackson', 'Huron', 'Holmes', 'Hocking', 'Highland', 'Henry', 'Harrison', 'Hardin', 'Hancock', 'Hamilton', 'Guernsey', 'Greene', 'Geauga', 'Gallia', 'Fulton', 'Franklin', 'Fayette', 'Fairfield', 'Erie', 'Delaware', 'Defiance', 'Darke', 'Cuyahoga', 'Crawford', 'Coshocton', 'Columbiana', 'Clinton', 'Clermont', 'Clark', 'Champaign', 'Carroll', 'Butler', 'Brown', 'Belmont', 'Auglaize', 'Athens', 'Ashtabula', 'Ashland', 'Allen', 'Adams']


In [21]:
L = []


 
for county in counties:
    url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"

    ts = TS()
    ts.loads(url)
    dashboard = ts.getWorkbook()


    countryDashboard = ts.getWorksheet("New Map").select("county", county)

    raceDashboard = dashboard.setParameter('Key Metrics', "Race")

    df = raceDashboard.getWorksheet('New Demographics').data

    df_dict = process_county(county, df)
    L.append(df_dict)

    print('Success\t\t', county)
        
# len(L)
Ohio = pd.DataFrame(L)
print(Ohio.shape)
Ohio.head()

Success		 Wyandot
Success		 Wood
Success		 Williams
Success		 Wayne
Success		 Washington
Success		 Warren
Success		 Vinton
Success		 Van Wert
Success		 Union
Success		 Tuscarawas
Success		 Trumbull
Success		 Summit
Success		 Stark
Success		 Shelby
Success		 Seneca
Success		 Scioto
Success		 Sandusky
Success		 Ross
Success		 Richland
Success		 Putnam
Success		 Preble
Success		 Portage
Success		 Pike
Success		 Pickaway
Success		 Perry
Success		 Paulding
Success		 Ottawa
Success		 Noble
Success		 Muskingum
Success		 Morrow
Success		 Morgan
Success		 Montgomery
Success		 Monroe
Success		 Miami
Success		 Mercer
Success		 Meigs
Success		 Medina
Success		 Marion
Success		 Mahoning
Success		 Madison
Success		 Lucas
Success		 Lorain
Success		 Logan
Success		 Licking
Success		 Lawrence
Success		 Lake
Success		 Knox
Success		 Jefferson
Success		 Jackson
Success		 Huron
Success		 Holmes
Success		 Hocking
Success		 Highland
Success		 Henry
Success		 Harrison
Success		 Hardin
Success		 Hancock
Succe

,County,Numb_White,Rate_White,Numb_Unknown,Rate_Unknown,Numb_Other,Rate_Other,Numb_Black or African American,Rate_Black or African American,Numb_Asian,Rate_Asian,Numb_American Indian or Alaska Native,Rate_American Indian or Alaska Native,Numb_Native Hawaiian or Other Pacific Islander,Rate_Native Hawaiian or Other Pacific Islander,Numb_Multiracial,Rate_Multiracial
0,Wyandot,6124,0.288664,970,%null%,357,%null%,14,0.160920,41,0.267974,10.0,0.153846,7.0,0.875000,67,0.274590
1,Wood,50041,0.410997,2097,%null%,3641,%null%,950,0.256479,1383,0.572197,137.0,0.320093,42.0,0.724138,376,0.153157
2,Williams,8029,0.226565,269,%null%,2978,%null%,29,0.060417,50,0.238095,9.0,0.068182,5.0,0.454545,218,0.517815
3,Wayne,31374,0.284760,1492,%null%,1351,%null%,487,0.257128,513,0.407143,47.0,0.192623,18.0,0.360000,182,0.087290
4,Washington,18617,0.323977,1725,%null%,579,%null%,161,0.205619,113,0.274939,25.0,0.147929,8.0,0.571429,60,0.056075


In [22]:

# print(len(L))

import numpy as np

state = 'Ohio' ## 

new_L = []

for idx, row in Ohio.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    new_d['Rate_White'] = d['Rate_White']
    
    new_d['Numb_Black'] = d['Numb_Black or African American']
    new_d['Rate_Black'] = d['Rate_Black or African American']
    
    new_d['Numb_Asian'] = d['Numb_Asian']
    new_d['Rate_Asian'] = d['Rate_Asian']
    
    new_d['Numb_AIndA'] = d['Numb_American Indian or Alaska Native']
    new_d['Rate_AIndA'] = d['Rate_American Indian or Alaska Native']
    
    
    new_L.append(new_d)

    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian', 'AIndA']:

    Rate = T['Rate_' + Race] 
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(88, 24)


,State,County,Geographic Area Name,Numb_AIndA,Numb_Asian,Numb_Black,Numb_White,Rate_AIndA,Rate_AIndAC,Rate_Asian,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Ohio,Wyandot,"Wyandot, Ohio",10.0,41,14,6124,0.153846,0.153846,0.267974,...,0.288664,65,158,153,190,87,210,21215,21452,21772
1,Ohio,Wood,"Wood, Ohio",137.0,1383,950,50041,0.320093,0.320093,0.572197,...,0.410997,428,1187,2417,3082,3704,4931,121755,124089,130817
2,Ohio,Williams,"Williams, Ohio",9.0,50,29,8029,0.068182,0.068182,0.238095,...,0.226565,132,316,210,280,480,678,35438,35837,36692
3,Ohio,Wayne,"Wayne, Ohio",47.0,513,487,31374,0.192623,0.192623,0.407143,...,0.284760,244,871,1260,1660,1894,3082,110177,112174,115710
4,Ohio,Washington,"Washington, Ohio",25.0,113,161,18617,0.147929,0.147929,0.274939,...,0.323977,169,655,411,618,783,1278,57464,58470,59911


In [23]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,88.0,0.188536,0.064120,0.022989,0.147228,0.191090,0.230360,0.400367
1,Rate_White,88.0,0.306695,0.070676,0.111760,0.261415,0.288521,0.349820,0.487740
2,Rate_Parity,88.0,0.118159,0.057522,-0.123534,0.083596,0.122280,0.156292,0.248870


## D. South Carolina* (Done)

In [36]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
ts = TS()
ts.loads(url)
ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")
ws.data

,Recipient County for maps-value,Recipient County for maps-alias,Recipient Zip Code for maps-alias,Latitude (generated)-value,Latitude (generated)-alias,Longitude (generated)-value,Longitude (generated)-alias,AGG(Count individuals with Suppression )-alias,AGG(Count individuals with Suppression for bar graph)-alias
0,York,York,29945,32.6803,32.680,-80.7941,-80.794,"1,404","1,404"
1,York,York,29944,32.7909,32.791,-81.0332,-81.033,"1,527","1,527"
2,York,York,29943,32.4596,32.460,-81.1515,-81.152,127,127
3,York,York,29941,32.5534,32.553,-80.8144,-80.814,368,368
4,York,York,29940,32.5557,32.556,-80.6994,-80.699,"1,069","1,069"
...,...,...,...,...,...,...,...,...,...
592,Abbeville,Abbeville,0,0,0,0,0,0,0
593,Abbeville,Abbeville,0,0,0,0,0,0,0
594,Abbeville,Abbeville,0,0,0,0,0,0,0
595,Abbeville,Abbeville,0,0,0,0,0,0,0


In [37]:
import time 

def process_county(county, df):
    df = df.replace(-1, 0)
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for k, racedf in df.groupby('Assigned Race-value'):
        Numb = racedf['AGG(Count individuals with Suppression )-alias'].sum()
        if 'Asian' in k:
            k = 'Asian'
        # print(k)
        # print(Numb)å
        D['Numb_' + k] = Numb
    return D


In [38]:
county = 'Williamsburg'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)

process_county(county, df)

{'County': 'Williamsburg',
 'Numb_Asian': 18,
 'Numb_Black': 5385,
 'Numb_Other': 1356,
 'Numb_Unknown': 560,
 'Numb_White': 2855}

In [39]:
county = 'York'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)
process_county(county, df)

{'County': 'York',
 'Numb_Asian': 2361,
 'Numb_Black': 10219,
 'Numb_Other': 8709,
 'Numb_Unknown': 5921,
 'Numb_White': 50617}

In [40]:


def get_SouthCarolina_Raw():
    url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
    ts = TS()
    ts.loads(url)
    ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")

    # show filter columns and values
    filters = ws.getFilters()
    counties = next(iter([
        t["values"]
        for t in filters
        if t["column"] == "Recipient County for maps"
    ]))

    L = []
    for county in counties:
        
        wb = ws.setFilter('Recipient County for maps', county)
        countyWs = wb.getWorksheet("Final Age xSex x Race REC")

        # print(countyWs.data)
        df = countyWs.data
        d = process_county(county, df)
        L.append(d)
        print('Success\t\t', county)
        time.sleep(0.5)
        
    SouthCarolina = pd.DataFrame(L)
    return SouthCarolina



# SouthCarolina = southcarolina.get_SouthCarolina_Raw()
SouthCarolina = get_SouthCarolina_Raw()
SouthCarolina = SouthCarolina.replace(0, np.nan)
print(SouthCarolina.shape)
SouthCarolina.head()

Success		 Abbeville
Success		 Aiken
Success		 Allendale
Success		 Anderson
Success		 Bamberg
Success		 Barnwell
Success		 Beaufort
Success		 Berkeley
Success		 Calhoun
Success		 Charleston
Success		 Cherokee
Success		 Chester
Success		 Chesterfield
Success		 Clarendon
Success		 Colleton
Success		 Darlington
Success		 Dillon
Success		 Dorchester
Success		 Edgefield
Success		 Fairfield
Success		 Florence
Success		 Georgetown
Success		 Greenville
Success		 Greenwood
Success		 Hampton
Success		 Horry
Success		 Jasper
Success		 Kershaw
Success		 Lancaster
Success		 Laurens
Success		 Lee
Success		 Lexington
Success		 Marion
Success		 Marlboro
Success		 McCormick
Success		 nan
Success		 Newberry
Success		 Oconee
Success		 Orangeburg
Success		 Pickens
Success		 Richland
Success		 Saluda
Success		 Spartanburg
Success		 Sumter
Success		 Union
Success		 Williamsburg
Success		 York
(47, 6)


,County,Numb_Asian,Numb_Black,Numb_Other,Numb_Unknown,Numb_White
0,Abbeville,NaN,1807,1105,217,4641
1,Aiken,729.0,8160,5657,3056,31483
2,Allendale,10.0,1599,258,102,680
3,Anderson,786.0,6189,6920,4467,41154
4,Bamberg,5.0,1970,406,195,1549


In [42]:

def standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION):
    
    state = 'South Carolina' ## 

    new_L = []

    for idx, row in SouthCarolina.iterrows(): ## 
        d = row.to_dict()

        new_d = {}
        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])

        RenameCounties = {
            'Mckean': 'McKean',
            'Mccormick': 'McCormick'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
        
        new_d['Numb_White'] = d['Numb_White']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Numb_Black']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Numb_Asian']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Numb_Other']
        new_d['Rate_Other'] = np.nan

        new_d['Date'] = np.nan

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate

        Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race + 'C'] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    # print(T.shape)
    # T.head()
    return Data, T


In [43]:
state = 'South Carolina'
Data, T = standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Nan, South Carolina']
(46, 26)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Other,Numb_White,Rate_AIndA,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,South Carolina,Abbeville,"Abbeville, South Carolina",NaN,NaN,NaN,1807,1105,4641,NaN,...,0.264716,85,196,89,129,6776,7014,17223,17532,24527
1,South Carolina,Aiken,"Aiken, South Carolina",NaN,NaN,729.0,8160,5657,31483,NaN,...,0.253644,965,2325,2122,2882,43193,45318,120735,124123,170872
2,South Carolina,Allendale,"Allendale, South Carolina",NaN,NaN,10.0,1599,258,680,NaN,...,0.305206,36,71,74,82,6329,6378,2167,2228,8688
3,South Carolina,Anderson,"Anderson, South Carolina",NaN,NaN,786.0,6189,6920,41154,NaN,...,0.247219,689,1590,2111,2683,32960,35359,163136,166468,202558
4,South Carolina,Bamberg,"Bamberg, South Carolina",NaN,NaN,5.0,1970,406,1549,NaN,...,0.285741,60,123,76,99,8478,8562,5292,5421,14066


In [44]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
# STATE_VACCINE_TMP = {}
# STATE_VACCINE_TMP[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio', 'South Carolina']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,46.0,0.224074,0.037019,0.164259,0.195289,0.218252,0.247610,0.326283
1,Rate_White,46.0,0.281471,0.058483,0.184333,0.238799,0.268957,0.310273,0.505516
2,Rate_Parity,46.0,0.057397,0.051072,-0.025268,0.015043,0.053247,0.084811,0.179233


## D. Oregon* (Done)

In [45]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

ts = TS()
ts.loads(url)

In [46]:
worksheet = ts.getWorksheet("County Map Per Capita new")

SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}

counties = SelectionDict['Recip Address County']


def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Demographic Category-value']
        data = d['AGG(People Count Display)-alias']
        data = float(data.replace(',','')) if data!= 'Suppressed' else np.nan
        D['Numb_' + race] = data
    return D

county = 'COOS'

dashboard = worksheet.select('Recip Address County', county)
df = dashboard.getWorksheet('County - Race-Ethnicity').data
d = process_county(county, df)
d

{'County': 'COOS',
 'Numb_Unknown': 1460.0,
 'Numb_Other Race': 779.0,
 'Numb_White': 19187.0,
 'Numb_NHPI': 1348.0,
 'Numb_Hispanic': 623.0,
 'Numb_Black': 105.0,
 'Numb_Asian': 317.0,
 'Numb_AI/AN': 773.0}

In [47]:
L = []


url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

ts = TS()
ts.loads(url)

worksheet = ts.getWorksheet("County Map Per Capita new")


        
for county in counties:
    dashboard = worksheet.select('Recip Address County', county)
    df = dashboard.getWorksheet('County - Race-Ethnicity').data
    df_dict = process_county(county, df)
    L.append(df_dict)
    print('Success\t\t', county)

Oregon = pd.DataFrame(L)
print(Oregon.shape)
Oregon.head()

Success		 YAMHILL
Success		 WHEELER
Success		 WASHINGTON
Success		 WASCO
Success		 WALLOWA
Success		 UNION
Success		 UMATILLA
Success		 TILLAMOOK
Success		 SHERMAN
Success		 POLK
Success		 MULTNOMAH
Success		 MORROW
Success		 MARION
Success		 MALHEUR
Success		 LINN
Success		 LINCOLN
Success		 LANE
Success		 LAKE
Success		 KLAMATH
Success		 JOSEPHINE
Success		 JEFFERSON
Success		 JACKSON
Success		 HOOD RIVER
Success		 HARNEY
Success		 GRANT
Success		 GILLIAM
Success		 DOUGLAS
Success		 DESCHUTES
Success		 CURRY
Success		 CROOK
Success		 COOS
Success		 COLUMBIA
Success		 CLATSOP
Success		 CLACKAMAS
Success		 BENTON
Success		 BAKER
(36, 9)


,County,Numb_Unknown,Numb_Other Race,Numb_White,Numb_NHPI,Numb_Hispanic,Numb_Black,Numb_Asian,Numb_AI/AN
0,YAMHILL,4761.0,2009.0,27595.0,162.0,4040.0,197.0,604.0,1033.0
1,WHEELER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WASHINGTON,18226.0,21711.0,155561.0,2945.0,24006.0,4497.0,28330.0,3401.0
3,WASCO,1285.0,158.0,7370.0,43.0,1317.0,36.0,111.0,175.0
4,WALLOWA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# print(len(L))

import numpy as np

state = 'Oregon' ## 
new_L = []


for idx, row in Oregon.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['Numb_White'] if d['Numb_White']!= '-' else np.nan
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black'] if d['Numb_Black']!= '-' else np.nan
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Numb_Asian'] if d['Numb_Asian']!= '-' else np.nan
    new_d['Rate_Asian'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

(36, 22)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_AsianC,Rate_Black,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Oregon,Yamhill,"Yamhill, Oregon",NaN,604.0,197.0,27595.0,0.278984,0.178804,0.153069,...,0.273256,2148,3729,2165,3378,1287,2014,97848,100986,107100
1,Oregon,Wheeler,"Wheeler, Oregon",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,34,57,14,30,3,18,1224,1276,1332
2,Oregon,Washington,"Washington, Oregon",NaN,28330.0,4497.0,155561.0,0.402318,0.332824,0.295972,...,0.308449,6827,13719,70417,85120,15194,22100,479049,504333,601592
3,Oregon,Wasco,"Wasco, Oregon",NaN,111.0,36.0,7370.0,0.370000,0.236170,0.167442,...,0.295557,1003,1407,300,470,215,352,24271,24936,26682
4,Oregon,Wallowa,"Wallowa, Oregon",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,68,156,36,68,33,80,6890,7051,7208


In [49]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['New York', 'Wisconsin', 'Ohio', 'South Carolina', 'Oregon']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,25.0,0.264329,0.091124,0.089770,0.200000,0.256000,0.323782,0.424934
1,Rate_White,25.0,0.313887,0.067946,0.152325,0.273038,0.303655,0.373248,0.459779
2,Rate_Parity,25.0,0.049558,0.062680,-0.042761,0.016756,0.046015,0.072582,0.194271


# All in One

## Read Previous Vaccine Data

In [50]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [51]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [52]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [53]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

In [54]:
[i for i in STATE_VACCINE]

['New York', 'Wisconsin', 'Ohio', 'South Carolina', 'Oregon']

## Save Data

In [55]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Tableau.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-04-28_Tableau.p
If yes, please enter: Yes


 Yes


Successfully Saved!
